In [1]:
pip install sqlalchemy pymysql

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [12]:
import pandas as pd
import numpy as np
import mysql.connector
import math

In [13]:
# Connect to mysql 
conn = mysql.connector.connect(
    host="192.168.3.19",
    user="lighthouse",
    password="letmein",
    database="sample_db"
)

In [14]:
# Load and clean CSV
def load_and_clean_csv(filepath: str) -> pd.DataFrame:
    df = pd.read_csv(filepath)
    df.columns = (
        df.columns
            .str.strip()
            .str.replace(" ","")
    )
    date_cols = [
        "ApplicationDate",
        "DateEmailReceived",
        "ApprovedDate"
    ]
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce").dt.date

    bool_map = {
        "Yes": 1, "No": 2, "Unknown": 3,
        "Y": 1, "N": 2, "U": 3,
        True: 1, False: 2
    }
    bool_cols = ["Approved", "SampleSent", "ResultReceived"]
    for col in bool_cols:
        if col in df.columns:
            df[col] = df[col].map(bool_map)
            df[col] = df[col].astype(object).where(df[col].notna(), None)

    return df

In [15]:
#Force mysql safe values
def df_to_mysql_safe(df: pd.DataFrame) -> pd.DataFrame:
    df = df.astype(object)
    df = df.replace({np.nan: None})
    return df

In [16]:
# Load CSV data
csv_path = "applications.csv"
df_csv = load_and_clean_csv(csv_path)
df_csv = df_csv[["ApplicationID","ApplicationDate","DateEmailReceived","Approved", "ApprovedDate", "SampleSent", "ResultReceived"]]
df_csv = df_to_mysql_safe(df_csv)

df_csv.head()


,ApplicationID,ApplicationDate,DateEmailReceived,Approved,ApprovedDate,SampleSent,ResultReceived
0,P2131,2025-04-06,2025-04-06,1.0,2025-06-06,1.0,1.0
1,P2194,2025-01-07,2025-01-07,1.0,2025-02-07,1.0,1.0
2,P2033,None,None,1.0,None,1.0,1.0
3,P1235,None,None,1.0,None,1.0,1.0
4,P1998,None,None,1.0,None,1.0,1.0


In [17]:
# Load corresponding mysql data
def load_mysql_data(conn, application_ids):
    placeholders = ",".join(["%s"] * len(application_ids))
    sql_query = f"""
        SELECT
            ApplicationID,
            ApplicationDate,
            DateEmailReceived,
            Approved,
            ApprovedDate,
            SampleSent,
            ResultReceived
        FROM application
        WHERE ApplicationID IN ({placeholders})
    """
    return pd.read_sql(sql_query, conn, params=application_ids)

In [18]:
application_ids = df_csv["ApplicationID"].dropna().tolist()
df_db = load_mysql_data(conn, application_ids)
df_db = df_to_mysql_safe(df_db)

df_db.head()

C:\Users\PachawoBisani\AppData\Local\Temp\ipykernel_10696\1555535017.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql_query, conn, params=application_ids)


,ApplicationID,ApplicationDate,DateEmailReceived,Approved,ApprovedDate,SampleSent,ResultReceived
0,P1530,2024-05-08,2024-05-08 00:00:00,1,2024-12-08,1,1
1,P1205,2024-03-03,2024-08-03 00:00:00,1,None,1,1
2,P1205,2024-03-03,2024-08-03 00:00:00,1,None,1,1
3,P1187,2023-05-12,2023-06-12 00:00:00,2,2023-08-12,2,2
4,P1202,2023-12-15,2023-12-18 00:00:00,1,2023-12-19,1,1


In [19]:
#Remove duplicates
df_csv = df_csv.drop_duplicates(subset="ApplicationID", keep="first")
df_db = df_db.drop_duplicates(subset="ApplicationID", keep="first")

#ALight dataframes
df_csv = df_csv.set_index("ApplicationID").sort_index()
df_db  = df_db.set_index("ApplicationID").sort_index()


# Ensure same column order
df_db = df_db[df_csv.columns]



In [22]:
# Sanity check
if not df_csv.index.is_unique:
    dupes = df_csv.index[df_csv.index.duplicated()].unique()
    raise ValueError(f"Duplicate ApplicationIDs found: {list(dupes)}")

if not df_db.index.is_unique:
    dupes = df_db.index[df_db.index.duplicated()].unique()
    raise ValueError(f"Duplicate ApplicationIDs found: {list(dupes)}")

assert df_csv.index.is_unique
assert df_db.index.is_unique
assert list(df_csv.columns) == list(df_db.columns)

In [27]:
diff = df_csv.compare(df_db, keep_equal=False)
diff

ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects

In [25]:
def build_change_log(diff_df):
    changes = []

    for app_id in diff_df.index:
        for col in diff_df.columns.levels[0]:
            if col in diff_df.columns:
                changes.append({
                    "ApplicationID": app_id,
                    "Field": col,
                    "OldValue": diff_df.loc[app_id, (col, "other")],
                    "NewValue": diff_df.loc[app_id, (col, "self")]
                })

    return pd.DataFrame(changes)


In [26]:
change_log = build_change_log(diff)
change_log.head()

NameError: name 'diff' is not defined